### Trapezoidal Rule Integration: Python, Cython, and NumPy Implementation
`- Siva Sundar, EE23B151`

The aim of the code is to analyse the efficiency of **python** and **cython** codes, keeping **numpy** as reference. We check the time taken by each code to implement trapezoidal rule of integration, and also the results they produce for the same partitions. This data can be used to infer which code is faster as well as more accurate to actual analytical values.

We conduct the following tests against each code to see its efficiency and accuracy:
| Test No. | Function                     | Area using Calculus        |
|----------|------------------------------|---------------------------|
| 1        | $ y = x^2, \, x \in [0,1] $ |  $\frac{1}{3}$         |
| 2        | $ y = \sin(x), \, x \in [0,\pi] $ |  $2$                   |
| 3        | $ y = e^x, \, x \in [0,1] $ |  $e - 1 \approx 1.7183$ |
| 4        | $ y = \frac{1}{x}, \, x \in [1,2] $ |  $ \ln(2) \approx 0.6931$ |


**Task 1:** To define a **pure python** function `py_trapz()` implementing trapezoidal rule to find proper integral.

In [1]:
# Pure Python function
def py_trapz(f, a, b, n):
    h = (b-a)/float(n)
    sum = 0.5*h*(f(a)+f(b))
    for i in range(1,n):
        sum += h*f(a+i*h)    # Area of elementary trapezoid
    return sum

**Task 2:** To define **optimized cython** function `cy_trapz()` implementing trapezoidal rule to find proper integral.

In [2]:
%load_ext Cython

In [16]:
%%cython --annotate

import cython
from cython.cimports.libc import math           # Importing Math library from standard C libraries

# Optimized Cython Function
@cython.cdivision(True)
@cython.returns(float)
def cy_trapz(f, float a, float b, int n):
    cdef int i
    cdef float h = (b - a) / n
    cdef float sum = 0.0

    # Initialize sum using the trapezoidal rule
    sum = 0.5 * h * (f(a) + f(b))

    # Loop through the intervals
    for i in range(1, n):
        sum += h * f(a + i * h)
    return sum

# Cython variant of test functions
@cython.cdivision(True)
@cython.returns(float)
def cy_sqr(float x):
    """Return the square of x."""
    return math.pow(x,2)

@cython.cdivision(True)
@cython.returns(float)
def cy_inv(float x):
    """Return the inverse of x."""
    return 1.0 / x

@cython.cdivision(True)
@cython.returns(float)
def cy_sin(float x):
    """Return the sine of x."""
    return math.sin(x)

@cython.cdivision(True)
@cython.returns(float)
def cy_exp(float x):
    """Return the exponential of x."""
    return math.exp(x)

Content of stdout:
_cython_magic_7969200515b95bba180a2553fe5cc466c11a2602.c
   Creating library C:\Users\LENOVO\.ipython\cython\Users\LENOVO\.ipython\cython\_cython_magic_7969200515b95bba180a2553fe5cc466c11a2602.cp310-win_amd64.lib and object C:\Users\LENOVO\.ipython\cython\Users\LENOVO\.ipython\cython\_cython_magic_7969200515b95bba180a2553fe5cc466c11a2602.cp310-win_amd64.exp
Generating code
Finished generating code

**Task 3:** Using `trapz()` in numpy library to implement trapezoidal rule to get reference results.

In [4]:
import numpy as np
# Storing the actual results (reference is numpy.trapz, I divided intervals into 10 million parts) for later use (Error calculations)
results = []; num = 10000000
x = np.linspace(0, 1, num); y = (x)**2; results.append(np.trapz(y, x))
x = np.linspace(0, np.pi, num); y = np.sin(x); results.append(np.trapz(y, x))
x = np.linspace(0, 1, num); y = np.exp(x); results.append(np.trapz(y, x))
x = np.linspace(1, 2, num); y = 1/(x); results.append(np.trapz(y, x))
print(results)

[0.33333333333333504, 1.999999999999982, 1.718281828459046, 0.6931471805599465]


**Task 4:** To compare the **performance** and **accuracy** of the functions `py_trapz()`, `cy_trapz()`, `numpy.trapz()`.

For `py_trapz()`:

In [10]:
import math

def py_sqr(x):      # Square function
    return x**2

def py_inv(x):      # Inverse function
    return 1/x

num = 10000
print(f"Time analysis for pure python function 'py_trapz()' with {num} partitions:")

# Test 1
print("Test 1: ", end='')
%timeit py_trapz(py_sqr, 0, 1, num)
result = py_trapz(py_sqr, 0, 1, num)
print(f"""\tResult = {result}
\tError = {result - results[0]}, % Error = {(result - results[0])/results[0] * 100} %""")

# Test 2
print("Test 2: ", end='')
%timeit py_trapz(math.sin, 0, math.pi, num)
result = py_trapz(math.sin, 0, math.pi, num)
print(f"""\tResult = {result}
\tError = {result - results[1]}, % Error = {(result - results[1])/results[1] * 100} %""")

# Test 3
print("Test 3: ", end='')
%timeit py_trapz(math.exp, 0, 1, num)
result = py_trapz(math.exp, 0, 1, num)
print(f"""\tResult = {result}
\tError = {result - results[2]}, % Error = {(result - results[2])/results[2] * 100} %""")

# Test 4
print("Test 4: ", end='')
%timeit py_trapz(py_inv, 1, 2, num)
result = py_trapz(py_inv, 1, 2, num)
print(f"""\tResult = {result}
\tError = {result - results[3]}, % Error = {(result - results[3])/results[3] * 100} %""")

Time analysis for pure python function 'py_trapz()' with 10000 partitions:
Test 1: 2.77 ms ± 116 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
	Result = 0.3333333349999995
	Error = 1.6666644730989333e-09, % Error = 4.999993419296774e-07 %
Test 2: 1.56 ms ± 16.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
	Result = 1.999999983550664
	Error = -1.6449317907785144e-08, % Error = -8.224658953892646e-07 %
Test 3: 1.59 ms ± 26.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
	Result = 1.7182818298909466
	Error = 1.4319005980922839e-09, % Error = 8.33332794641966e-08 %
Test 4: 1.7 ms ± 15.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
	Result = 0.6931471811849486
	Error = 6.250020501141762e-10, % Error = 9.016873582451558e-08 %


For `cy_trapz()`:

In [6]:
num = 10000
print(f"Time analysis for optimized cython function 'cy_trapz()' with {num} partitions:")

# Test 1
print("Test 1: ", end='')
%timeit cy_trapz(cy_sqr, 0, 1, num)
result = cy_trapz(cy_sqr, 0, 1, num)
print(f"""\tResult = {result}
\tError = {result - results[0]}, % Error = {(result - results[0])/results[0] * 100} %""")

# Test 2
print("Test 2: ", end='')
%timeit cy_trapz(cy_sin, 0, math.pi, num)
result = cy_trapz(cy_sin, 0, math.pi, num)
print(f"""\tResult = {result}
\tError = {result - results[1]}, % Error = {(result - results[1])/results[1] * 100} %""")

# Test 3
print("Test 3: ", end='')
%timeit cy_trapz(cy_exp, 0, 1, num)
result = cy_trapz(cy_exp, 0, 1, num)
print(f"""\tResult = {result}
\tError = {result - results[2]}, % Error = {(result - results[2])/results[2] * 100} %""")

# Test 4
print("Test 4: ", end='')
%timeit cy_trapz(cy_inv, 1, 2, num)
result = cy_trapz(cy_inv, 1, 2, num)
print(f"""\tResult = {result}
\tError = {result - results[3]}, % Error = {(result - results[3])/results[3] * 100} %""")

Time analysis for optimized cython function 'cy_trapz()' with 10000 partitions:
Test 1: 1.03 ms ± 28 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
	Result = 0.33333322405815125
	Error = -1.0927518379055812e-07, % Error = -3.2782555137167266e-05 %
Test 2: 783 μs ± 135 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
	Result = 1.9999982118606567
	Error = -1.7881393252761058e-06, % Error = -8.940696626380609e-05 %
Test 3: 685 μs ± 20.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
	Result = 1.7182813882827759
	Error = -4.4017627010006777e-07, % Error = -2.5617233611486047e-05 %
Test 4: 725 μs ± 165 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
	Result = 0.6931471228599548
	Error = -5.7699991673487716e-08, % Error = -8.324349184667507e-06 %


For `numpy.trapz()`, we include the time taken for creating the x and y arrays also.

In [11]:
import numpy as np

num = 10000
print(f"Time analysis for numpy function 'numpy.trapz()' with {num} partitions:")

# Test 1
print("Test 1: ", end='')
%timeit x = np.linspace(0, 1, num); y = (x)**2; np.trapz(y,x)
x = np.linspace(0, 1, num)                  
y = (x)**2                                  # Again defined x and y as the timeit-
result = np.trapz(y,x)                      # -magic line would not update variables in it
print(f"""\tResult = {result}
\tError = {result - results[0]}, % Error = {(result - results[0])/results[0] * 100} %""")

# Test 2
print("Test 2: ", end='')
%timeit x = np.linspace(0, np.pi, num); y = np.sin(x); np.trapz(y,x)
x = np.linspace(0, np.pi, num)
y = np.sin(x)
result = np.trapz(y,x)
print(f"""\tResult = {result}
\tError = {result - results[1]}, % Error = {(result - results[1])/results[1] * 100} %""")

# Test 3
print("Test 3: ", end='')
%timeit x = np.linspace(0, 1, num); y = np.exp(x); np.trapz(y,x)
x = np.linspace(0, 1, num)
y = np.exp(x)
result = np.trapz(y,x)
print(f"""\tResult = {result}
\tError = {result - results[2]}, % Error = {(result - results[2])/results[2] * 100} %""")

# Test 4
print("Test 4: ", end='')
%timeit x = np.linspace(1, 2, num); y = 1/(x); np.trapz(y,x)
x = np.linspace(1, 2, num)
y = 1/(x)
result = np.trapz(y,x)
print(f"""\tResult = {result}
\tError = {result - results[3]}, % Error = {(result - results[3])/results[3] * 100} %""")


Time analysis for numpy function 'numpy.trapz()' with 10000 partitions:
Test 1: 62 μs ± 867 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
	Result = 0.3333333350003334
	Error = 1.6669983726735893e-09, % Error = 5.000995118020742e-07 %
Test 2: 124 μs ± 2.04 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
	Result = 1.999999983547369
	Error = -1.645261304972223e-08, % Error = -8.22630652486119e-07 %
Test 3: 125 μs ± 13.6 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
	Result = 1.7182818298912328
	Error = 1.4321868135880322e-09, % Error = 8.334993653936366e-08 %
Test 4: 73.3 μs ± 5.14 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
	Result = 0.6931471811850702
	Error = 6.251237305576751e-10, % Error = 9.018629060175648e-08 %


**Task 5**: Compare the **performance** for the three functions while integrating $f(x) = x^2$, $x \in [0,10]$ with **10 million partitions**.

In [9]:
num = 10000000
print("Time taken for:")
print("\t\u2022 py_trapz(): ", end="")
%timeit py_trapz(py_sqr, 0, 10, num)
print("\t\u2022 cy_trapz(): ", end="")
%timeit cy_trapz(cy_sqr, 0, 10, num)
print("\t\u2022 numpy.trapz(): ", end="")
%timeit x = np.linspace(0, 10, num); y = (x)**2; np.trapz(y,x)

Time taken for:
	• py_trapz(): 2.58 s ± 39.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
	• cy_trapz(): 943 ms ± 6.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
	• numpy.trapz(): 188 ms ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The results for **Task 4** is as follows:
| Test No. | Execution Time                                  | Result    | log<sub>10</sub>\|Error\|                               |
|----------|------------------------------------------------|-----------|------------------------------------------------|
|          | **Python: py_trapz()**                         |           |                                                |
|          |                                                |           |                                                |
| 1        | 2.77 ms                                        | 0.333     | -8.78 |
| 2        | 1.56 ms                                        | 2.000     | -7.78 |
| 3        | 1.59 ms                                        | 1.718     | -8.84 |
| 4        | 1.70 ms                                        | 0.693     | -9.20 |
|          | **Cython: cy_trapz()**                        |           |                                                |
|          |                                                |           |                                                |
| 1        | 1.03 ms                                        | 0.333     | -7.96 |
| 2        | 0.783 ms                                       | 2.000     | -5.75 |
| 3        | 0.685 ms                                       | 1.718     | -6.36 |
| 4        | 0.725 ms                                       | 0.693     | -7.24 |
|          | **NumPy: numpy.trapz()**                      |           |                                                |
|          |                                                |           |                                                |
| 1        | 62 µs                                          | 0.333     | -8.78 |
| 2        | 124 µs                                         | 2.000     | -7.78 |
| 3        | 125 µs                                         | 1.718     | -8.84 |
| 4        | 73.3 µs                                        | 0.693     | -9.20 |


The outputs are carefully analyzed, and the resulting insights and conclusions are documented in detail in the **README file**.